In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5
!pip install -r ./yolov5/requirements.txt  # install
!pip install -U albumentations #using image augmentation feature in yolov5

In [ ]:
import random
import numpy as np
import pandas as pd
import cv2
import os
import ast
import shutil
from IPython.display import display
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv", index_col="image_id")
train.head(10)

In [ ]:
img = cv2.imread("../input/tensorflow-great-barrier-reef/train_images/video_0/978.jpg")

start_pt = (825, 224)
end_pt = (875, 259)
color = (0, 0, 255)
thickness = 2
image = cv2.rectangle(img, start_pt, end_pt, color, thickness)
plt.imshow(image)

In [ ]:
!mkdir -p ./datasets/coral/labels
!mkdir -p ./datasets/coral/images

path_to_txt_folder = "./datasets/coral/labels"
final_path_to_images = "./datasets/coral/images"

def txt_data(path):
    for folder in os.listdir(path):
        path_to_folder = os.path.join(path, folder)
        _, video_id = folder.split("_")

        for image in os.listdir(path_to_folder):
            path_to_image = os.path.join(path_to_folder, image)

            image_id, img_ext = image.split(".")
            image_match = video_id + "-" + image_id

            img_temp = cv2.imread(path_to_image)
            height, width, channels = img_temp.shape

            image_data = train.loc[image_match]
            annotation_list = ast.literal_eval(image_data.annotations)

            if bool(annotation_list) == True:
                path_to_file = os.path.join(path_to_txt_folder, image_match + ".txt")
                file_ = open(path_to_file, "w")
                for i in annotation_list:
                    x_min = i['x']
                    y_min = i['y']
                    box_width = i['width']
                    box_height = i['height']
                    class_label=str(0)
                    x_min = str(round((x_min+(box_width/2))/width, 10))
                    y_min = str(round((y_min+(box_height/2))/height, 10))
                    box_width = str(round(box_width/width, 10))
                    box_height = str(round(box_height/height, 10))
                    file_.write(class_label +" "+ x_min +" "+ y_min +" "+ box_width +" "+ box_height + "\n")
                file_.close()
    
                image_new_name = os.path.join(final_path_to_images, image_match + "." + img_ext)
#             new_path_to_image = os.path.join(final_path_to_images, image_new_name)
                shutil.copyfile(path_to_image, image_new_name)

txt_data("../input/tensorflow-great-barrier-reef/train_images")

In [ ]:
img = cv2.imread("./datasets/coral/images/0-978.jpg")

start_pt = (825, 224)
end_pt = (875, 259)
color = (0, 0, 255)
thickness = 2
image = cv2.rectangle(img, start_pt, end_pt, color, thickness)
plt.imshow(image)

In [ ]:
!mkdir -p ./datasets/gbr/train/labels
!mkdir -p ./datasets/gbr/train/images
!mkdir -p ./datasets/gbr/val/labels
!mkdir -p ./datasets/gbr/val/images

file_list = os.listdir(final_path_to_images)
np.random.shuffle(file_list)
train_val_split = 0.9
train_len = round(train_val_split * len(file_list))
train_list = file_list[:train_len]
val_list = file_list[train_len:]

def split(new_file_path, typ_list=train_list):
    for file in typ_list:
        new_image_path = os.path.join(new_file_path, "images")
        image_path = os.path.join(final_path_to_images, file)
        shutil.move(image_path, new_image_path)
        
        indx, _ = file.split(".")
        new_label_path = os.path.join(new_file_path, "labels")
        label_path = os.path.join(path_to_txt_folder, indx + '.txt')
        shutil.move(label_path, new_label_path)
        
split("./datasets/gbr/train", train_list)
split("./datasets/gbr/val", val_list)
!rm -rf "./datasets/coral"

In [ ]:
shutil.copyfile("../input/coral-yaml/coral.yaml", "./yolov5/data/coral.yaml")
shutil.copyfile("../input/coral-yaml/hyp.yaml", "./yolov5/data/hyps/hyp.yaml")

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_key")

!pip install wandb
!wandb login $secret_value_0

In [ ]:
# Train YOLOv5m on coral_dataset for 3 epochs
!python ./yolov5/train.py --img 1280 --rect --batch 16 --epochs 25 --data ./yolov5/data/coral.yaml --hyp ./yolov5/data/hyps/hyp.yaml --weights yolov5m6.pt

In [ ]:
import torch
ckpt_path = "./yolov5/runs/train/exp/weights/best.pt"

model = torch.hub.load('../input/yolov5', 'custom',
                           path=ckpt_path,
                           source='local',
                       force_reload=True)

model.conf = 0.20
model.iou = 0.45

In [ ]:
def result_string_calculator(results_dataframe):
    if results_dataframe.shape[0] != 0:
        for index,row in results_dataframe.iterrows():
            x_center = row['xmin'] + ((row['xmax'] - row['xmin'])/2)
            y_center = row['ymin'] + ((row['ymax'] - row['ymin'])/2)
            height= row['ymax'] - row['ymin']
            width = row['xmax'] - row['xmin']
            confidence_score = round(row['confidence'], 3)
        output_string = "{} {} {} {} {} ".format(confidence_score,round(x_center),round(y_center),round(width),round(height))
    else:
        output_string = "0 0 0 0 0"
    return output_string

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:
for (pixel_array, sample_prediction_df) in iter_test:
    results = model(pixel_array)
    print(results)
    results_df=results.pandas().xyxy[0]
    sample_prediction_df['annotations'] =  result_string_calculator(results_df) # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions